In [1]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig

max_len = 384
configuration = BertConfig() 

In [10]:
train_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json"
train_path = keras.utils.get_file("train.json", train_data_url)

30288272/30288272 [==============================] - 6s 0us/step


In [ ]:
with open(train_path,encoding='utf8') as f:
    raw_train_data=json.load(f)
         



In [127]:
tokenized_context=tokenizer.encode("hi")
tokenized_context

Encoding(num_tokens=3, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [133]:
class SquadExample:
    def __init__(self, question, context, start_char_idx, answer_text, all_answers):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.all_answers = all_answers
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx

        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())

        end_char_idx=start_char_idx + len(answer)
        if end_char_idx>len(context):
            self.skip=True
            return

        is_char_in_ans=[0]*len(context)
        for idx in range(start_char_idx,end_char_idx):
            is_char_in_ans[idx]=1
        
        tokenized_context=tokenizer.encode(context)

        ans_token_idx=[]
        for idx, (start,end) in enumerate(tokenized_context.offsets):
            if(sum(is_char_in_ans[start:end])>0):
                ans_token_idx.append(idx)
        
        if len(ans_token_idx)==0:
            self.skip=True
            return
        
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        tokenized_question = tokenizer.encode(question)

        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        attention_mask = [1] * len(input_ids)
        
        padding_length = max_len - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_context.offsets

def create_squad_examples(raw_data):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
                answer_text = qa["answers"][0]["text"]
                all_answers = [_["text"] for _ in qa["answers"]]
                start_char_idx = qa["answers"][0]["answer_start"]
                squad_eg = SquadExample(
                    question, context, start_char_idx, answer_text, all_answers
                )
                squad_eg.preprocess()
                squad_examples.append(squad_eg)
    return squad_examples

train_squad_examples = create_squad_examples(raw_train_data)
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")



87599 training points created.


In [124]:
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
save_path = "bert_base_uncased/"
tokenizer = BertWordPieceTokenizer("C:\\Users\\a3318\\.cache\\huggingface\\hub\\models--google-bert--bert-base-uncased\\snapshots\\86b5e0934494bd15c9632b12f734a8a67f723594\\vocab.txt", lowercase=True)


In [79]:
encoder = TFBertModel.from_pretrained("bert-base-uncased", from_pt=True)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

c:\Users\a3318\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\a3318\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: [

In [136]:
from transformers import BertTokenizer, BertModel
import torch
from torch import nn

model = BertModel.from_pretrained("bert-base-uncased")

In [168]:
class BertQA(nn.Module):
    def __init__(self):
        super().__init__()
        self.model=BertModel.from_pretrained("bert-base-uncased")
        self.d1=nn.Linear(768,1)
        self.d2=nn.Linear(768,1)

    def forward(self,data):
        x= self.model(
        input_ids=torch.tensor(data[0])[:5],
        attention_mask=torch.tensor(data[1])[:5],
        token_type_ids=torch.tensor(data[2])[:5]
        ).last_hidden_state
        out1=self.d1(x).squeeze(-1)
        out2=self.d2(x).squeeze(-1)

        return ([out1,out2])


model1=BertQA()

In [169]:
xx=model1(x_train)

In [193]:
x=torch.stack([xx[0][:5], xx[1][:5]], axis=0)
x11=x.view(-1,384)
x11.shape

torch.Size([10, 384])

In [195]:
criterion(x11,y.view(-1))

tensor(5.9839, grad_fn=<NllLossBackward0>)